In [1054]:
import numpy as np
import heapq
import math
import os

In [1055]:
def computeMinPath(graph, numVertices):
  minPath = np.zeros((numVertices, 2))
  for i in range(numVertices):
    edgeWeights = graph[i]
    
    minEdge = np.min(edgeWeights)
    secondMinEdge = np.partition(edgeWeights, 1)[1]

    minPath[i] = [minEdge, secondMinEdge]

  return minPath
      
def getOnePath(graph, numVertices):
  path = np.array([0])
  cost = 0
  for i in range(1, numVertices):
    cost += graph[path[-1]][i]
    path = np.concatenate([path, [i]]).astype(int)
  cost += graph[path[-1]][0]
  path = np.concatenate([path, [0]]).astype(int)
  return path, cost

def bound(graph, minPath, nextVertice, prevNode=None):
  if prevNode is None:
    return np.sum(minPath) / 2
  
  cost = graph[prevNode.path[-1]][nextVertice]

  bound = prevNode.bound * 2

  if cost >= minPath[nextVertice][1]:
    bound -= minPath[nextVertice][1]
    bound += cost
  
  if cost >= minPath[prevNode.path[-1]][1]:
    bound -= minPath[prevNode.path[-1]][1]
    bound += cost

  return bound / 2


## Branch and Bound

In [1056]:
class Node:
  def __init__(self, graph, minPath, prevNode=None, nextVertice=None):
    if prevNode is None or nextVertice is None:
      self.level = 1
      self.path = [0]
      self.cost = 0
      self.bound = bound(graph, minPath, 0)
      return

    self.level = prevNode.level + 1
    self.path = np.concatenate([prevNode.path, [nextVertice]]).astype(int)
    if len(prevNode.path) == 0:
      self.cost = 0
    else:
      self.cost = prevNode.cost + graph[prevNode.path[-1]][nextVertice]
    self.bound = bound(graph, minPath, nextVertice, prevNode)

  def __lt__(self, other):
    return self.bound < other.bound
  
  def __gt__(self, other):
    return self.bound > other.bound
  
  def __eq__(self, other):
    return self.bound == other.bound
  
  def __str__(self) -> str:
    return f"Path: {self.path} Cost: {self.cost} Bound: {self.bound}"

In [1057]:
def branchAndBoundTSP(graph, numVertices):
  minPath = computeMinPath(graph, numVertices)

  root = Node(graph, minPath, None, 0)
  queue = [root]
  heapq.heapify(queue)

  solution, best = getOnePath(graph, numVertices)

  while queue:
    node = heapq.heappop(queue)
    
    if node.level >= numVertices:
      finalNode = Node(graph, minPath, node, 0)
      if finalNode.cost < best:
        best = finalNode.cost
        solution = finalNode.path
    
    elif node.bound < best:
      unvisited = np.setdiff1d(np.arange(numVertices), node.path)
      for k in unvisited:
        newNode = Node(graph, minPath, node, k)
        if newNode.bound < best and graph[node.path[-1]][k] != np.inf:
          heapq.heappush(queue, newNode)

  return solution, best

## Twice Around the Tree

In [1058]:
def generateMST(graph, numVertices):
  included = [False] * numVertices
  included[0] = True

  cost = 0
  edges = []
  numVisited = 1

  while numVisited < numVertices:
    minEdge = (float('inf'), None, None) 

    for i in range(numVertices):
      if included[i]:
        for j in range(numVertices):
          if not included[j] and graph[i][j] < minEdge[0]:
            minEdge = (graph[i][j], i, j)

    cost, startVertex, endVertex = minEdge
    included[endVertex] = True
    cost += cost
    edges.append((startVertex, endVertex))
    numVisited += 1

  mstAdjMatrix = np.full((numVertices, numVertices), np.inf)
  for edge in edges:
    mstAdjMatrix[edge[0]][edge[1]] = mstAdjMatrix[edge[1]][edge[0]] = graph[edge[0]][edge[1]]

  return mstAdjMatrix, cost

In [1059]:
def preorderWalkAux(adjMatrix, startVertex, visited, order):
  order.append(startVertex)
  visited[startVertex] = True

  for neighbor in range(len(adjMatrix)):
    if adjMatrix[startVertex][neighbor] != np.inf and not visited[neighbor]:
      preorderWalkAux(adjMatrix, neighbor, visited, order)

def preorderWalk(adjMatrix, start_vertex=0):
  numVertices = len(adjMatrix)
  visited = [False] * numVertices
  order = []

  preorderWalkAux(adjMatrix, start_vertex, visited, order)

  return order

In [1060]:
def approximateTwiceAroundTheTree(graph, numVertices):
  mstAdjMatrix, mstCost = generateMST(graph, numVertices)

  preorder = preorderWalk(mstAdjMatrix)
  tour = preorder + [preorder[0]]

  tourCost = sum(graph[tour[i]][tour[i + 1]] for i in range(len(tour) - 1))

  return tour, tourCost

## Christofides

In [1061]:
def findMinimumWeightMatching(graph, oddVertices):
  if oddVertices is None:
    oddVertices = range(len(graph))

  edges = []
  for i in oddVertices:
    for j in oddVertices:
      if i < j and graph[i][j] != np.inf:
        edges.append((i, j, graph[i][j]))

  edges.sort(key=lambda x: x[2])

  matching = []
  visited = np.zeros(len(graph), dtype=bool)

  for edge in edges:
    if not visited[edge[0]] and not visited[edge[1]]:
      matching.append((edge[0], edge[1]))
      visited[edge[0]] = True
      visited[edge[1]] = True

  return matching

In [1062]:
def getEulerianCircuit(graph):
  startVertex = 0  # Choose a starting vertex
  numVertices = len(graph)
  visited = np.zeros(numVertices, dtype=bool)
  tour = []
  stack = np.array([startVertex])

  while stack.size > 0:
    vertex = stack[-1]
    if not visited[vertex]:
      visited[vertex] = True
      tour.append(vertex)
    unvisited_neighbors = np.where((graph[vertex] != np.inf) & (~visited))[0]
    if unvisited_neighbors.size > 0:
      next_vertex = unvisited_neighbors[0]
      stack = np.append(stack, next_vertex)
    else:
      stack = stack[:-1]

  return tour

In [1063]:
def christofides(graph, numVertices):
  mstAdjMatrix, mstCost = generateMST(graph, numVertices)

  oddVertices = [vertex for vertex, degree in enumerate(np.sum(mstAdjMatrix != np.inf, axis=1)) if degree % 2 != 0]

  matchingEdges = findMinimumWeightMatching(graph, oddVertices)

  multigraph = mstAdjMatrix.copy()
  for edge in matchingEdges:
    multigraph[edge[0]][edge[1]] = multigraph[edge[1]][edge[0]] = graph[edge[0]][edge[1]]

  eulerianCircuit = getEulerianCircuit(multigraph)
  hamiltonianCircuit = list(dict.fromkeys(eulerianCircuit))
  hamiltonianCircuit.append(hamiltonianCircuit[0])

  tourCost = sum(graph[hamiltonianCircuit[i]][hamiltonianCircuit[i + 1]] for i in range(len(hamiltonianCircuit) - 1))

  return hamiltonianCircuit, tourCost

## Reading input files

In [1064]:
def readCoordinatesFromFile(filename):
  coordinates = []

  with open(filename, 'r') as file:
    for line in file:
      if "NODE_COORD_SECTION" in line:
        break

    for line in file:
      if line.strip() == "EOF":
        break
      id, x, y = map(float, line.split())
      coordinates.append((int(id)-1, x, y))

  return coordinates

def calculateEuclideanDistance(point1, point2):
    x1, y1 = point1[1], point1[2]
    x2, y2 = point2[1], point2[2]
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def getGraphFromFile(filename):
    coordinates = readCoordinatesFromFile(filename)
    numPoints = len(coordinates)

    graph = np.full((numPoints, numPoints), np.inf)

    for i in range(numPoints):
      for j in range(i+1, numPoints):
        distance = calculateEuclideanDistance(coordinates[i], coordinates[j])
        graph[coordinates[i][0]][coordinates[j][0]] = graph[coordinates[j][0]][coordinates[i][0]] = distance

    return graph

In [1065]:
def getFilesInFolder(folderPath):
  return [os.path.join(folderPath, file) for file in os.listdir(folderPath) if os.path.isfile(os.path.join(folderPath, file))]

files = getFilesInFolder("data")

## Testing

In [1066]:
numVertices = 4
graph = np.full((numVertices, numVertices), np.inf)
graph = np.array([
       [0, 10, 15, 20],
       [10, 0, 35, 25],
       [15, 35, 0, 30],
       [20, 25, 30, 0]
       ])
solution, best = branchAndBoundTSP(graph, numVertices)

print(f"Solution: {solution} Cost: {best}")

Solution: [0 1 3 2 0] Cost: 80


In [1067]:
for file in files:
  graph = getGraphFromFile(file)
  numVertices = len(graph)
  # solution, best = branchAndBoundTSP(graph, numVertices)
  # print(f"Solution Branch and Bound: {solution} Cost: {best}")

  solution, best = approximateTwiceAroundTheTree(graph, numVertices)
  print(f"Solution Twice Around the Tree: {solution} Cost: {best}")

  solution, best = christofides(graph, numVertices)
  print(f"Solution Christofides: {solution} Cost: {best}")

  print("\n")

Solution Twice Around the Tree: [0, 62, 5, 48, 89, 9, 71, 20, 73, 10, 14, 16, 31, 46, 90, 44, 97, 22, 59, 61, 85, 26, 11, 19, 56, 6, 8, 86, 50, 60, 24, 80, 68, 63, 39, 53, 72, 49, 43, 1, 67, 84, 38, 29, 95, 77, 4, 36, 32, 12, 94, 81, 75, 51, 57, 27, 66, 92, 54, 82, 33, 28, 45, 2, 13, 70, 40, 99, 47, 42, 34, 76, 58, 83, 91, 7, 41, 88, 30, 79, 55, 96, 3, 64, 25, 65, 74, 18, 52, 78, 17, 23, 37, 35, 98, 87, 15, 21, 69, 93, 0] Cost: 30516.94176196444
Solution Christofides: [0, 62, 5, 48, 89, 9, 71, 20, 73, 10, 14, 16, 31, 46, 90, 44, 97, 22, 59, 61, 85, 26, 11, 19, 56, 6, 8, 86, 50, 60, 24, 80, 68, 63, 39, 53, 1, 43, 49, 72, 67, 84, 38, 29, 95, 77, 4, 36, 32, 12, 94, 81, 98, 35, 37, 23, 17, 78, 52, 18, 74, 96, 3, 64, 25, 65, 55, 79, 30, 88, 41, 7, 91, 69, 21, 15, 87, 93, 75, 51, 57, 27, 66, 92, 47, 99, 70, 13, 2, 42, 45, 28, 33, 82, 54, 40, 34, 76, 58, 83, 0] Cost: 37552.46971977364


Solution Twice Around the Tree: [0, 7, 21, 3, 17, 16, 1, 2, 15, 12, 11, 6, 5, 18, 9, 8, 10, 19, 20, 13, 14,